## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_final_count.json")

In [5]:
df_lookup.count()

12361

In [6]:
df_april = spark.read.json("/user/sprenner/training_official_april_hourly_cleaned_datasets.json")

In [7]:
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["datasetScope"], df_april["dataset"]))
df_april.createOrReplaceTempView("april")

In [8]:
df_joined = df_april.join(df_lookup, df_april.join_key_april == df_lookup.join_key_lookup, 'left_outer')

## add column hour

In [9]:
#column hour Already included from pipeline before!
#df_joined = df_joined.withColumn("hour", from_unixtime(df_joined['traceTimeentryUnix'], 'yyyy-MM-dd HH'))

## check

In [10]:
df_joined.createOrReplaceTempView("joined")

## create index column to later join the partition table correctly

In [11]:
df_joined_indexed = df_joined.sort("traceTimeentryUnix", ascending=True)
df_joined_indexed.createOrReplaceTempView("joined_indexed")
df_joined_indexed = spark.sql("select ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as id, * FROM joined_indexed")

In [12]:
df_joined_indexed.createOrReplaceTempView("joined_indexed")

## partition by 2 columns to count amount of files already downloaded in every dataset (uuid alone not enough, collisions).
## only count successful downloads to avoid >100% (STILL POSSIBLE if user downloads data twice)

In [13]:
#https://www.sqlservercentral.com/Forums/573706/ROWNUMBER-and-possibly-group-by
#https://stackoverflow.com/questions/6127338/sql-mysql-select-distinct-unique-but-return-all-columns
df_proportion = spark.sql("SELECT ROW_NUMBER() OVER (PARTITION BY hour, uuid, join_key_april ORDER BY traceTimeentryUnix ASC) AS Seq , *\
                           FROM (SELECT * FROM \
                                    (SELECT *, ROW_NUMBER() OVER(PARTITION BY hour, uuid, join_key_april, filename ORDER BY traceTimeentryUnix ASC) AS row_number\
                                     FROM joined_indexed WHERE clientState == 'DONE' AND join_key_lookup IS NOT NULL)\
                                 WHERE row_number == 1) \
                           ORDER BY traceTimeentryUnix ASC")
df_proportion.limit(50).toPandas()

Seq   id   account  accountHash clientState  clientStateHash  \
0   1    2    yliu      1494332205   DONE       -878527175         
1   1    4    changqia  224190018    DONE       -878527175         
2   2    7    yliu      1494332205   DONE       -878527175         
3   1    8    heng      99410464     DONE       -878527175         
4   3    9    yliu      1494332205   DONE       -878527175         
5   4    10   yliu      1494332205   DONE       -878527175         
6   5    11   yliu      1494332205   DONE       -878527175         
7   6    14   yliu      1494332205   DONE       -878527175         
8   7    18   yliu      1494332205   DONE       -878527175         
9   2    19   heng      99410464     DONE       -878527175         
10  8    21   yliu      1494332205   DONE       -878527175         
11  9    22   yliu      1494332205   DONE       -878527175         
12  10   23   yliu      1494332205   DONE       -878527175         
13  1    29   changqia  224190018    DONE       -878527175         
14  1    31   ssolomon  1221542610   DONE       -878527175         
15  1    37   ssolomon  1221542610   DONE       -878527175         
16  1    38   yliu      1494332205   DONE       -878527175         
17  2    39   yliu      1494332205   DONE       -878527175         
18  1    41   ssolomon  1221542610   DONE       -878527175         
19  3    42   yliu      1494332205   DONE       -878527175         
20  1    44   ssolomon  1221542610   DONE       -878527175         
21  4    45   yliu      1494332205   DONE       -878527175         
22  5    49   yliu      1494332205   DONE       -878527175         
23  6    50   yliu      1494332205   DONE       -878527175         
24  1    52   heng      99410464     DONE       -878527175         
25  7    53   yliu      1494332205   DONE       -878527175         
26  2    55   changqia  224190018    DONE       -878527175         
27  1    57   yliu      1494332205   DONE       -878527175         
28  1    59   changqia  224190018    DONE       -878527175         
29  2    64   changqia  224190018    DONE       -878527175         
30  3    70   changqia  224190018    DONE       -878527175         
31  1    76   ssolomon  1221542610   DONE       -878527175         
32  2    78   ssolomon  1221542610   DONE       -878527175         
33  3    80   ssolomon  1221542610   DONE       -878527175         
34  4    82   ssolomon  1221542610   DONE       -878527175         
35  5    83   ssolomon  1221542610   DONE       -878527175         
36  6    86   ssolomon  1221542610   DONE       -878527175         
37  7    88   ssolomon  1221542610   DONE       -878527175         
38  8    90   ssolomon  1221542610   DONE       -878527175         
39  9    92   ssolomon  1221542610   DONE       -878527175         
40  10   94   ssolomon  1221542610   DONE       -878527175         
41  11   96   ssolomon  1221542610   DONE       -878527175         
42  12   98   ssolomon  1221542610   DONE       -878527175         
43  13   100  ssolomon  1221542610   DONE       -878527175         
44  14   102  ssolomon  1221542610   DONE       -878527175         
45  15   104  ssolomon  1221542610   DONE       -878527175         
46  16   106  ssolomon  1221542610   DONE       -878527175         
47  17   108  ssolomon  1221542610   DONE       -878527175         
48  18   110  ssolomon  1221542610   DONE       -878527175         
49  19   112  ssolomon  1221542610   DONE       -878527175         

                                                                                                                                           dataset  \
0   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1   data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
2   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid115680

In [14]:
df_proportion.count()
#ohne join_key_lookup IS NOT NULL: 579934, mit der beschränkung: 578390
# mit : 578390
# werden mit hour evtl mehr weil weniger werte als duplikate gezählt werden?: 579384

579384

In [15]:
df_proportion_percent = df_proportion.withColumn("df_percentage", df_proportion['Seq']/df_proportion['count'])
#df_proportion_percent.limit(50).toPandas()

In [16]:
df_proportion_percent.createOrReplaceTempView("percent")
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()
#sollte nichts gefunden werden

Empty DataFrame
Columns: [Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_april, count, join_key_lookup, row_number, df_percentage]
Index: []

In [17]:
#spark.sql("SELECT * FROM percent WHERE id == 893353199188").toPandas()

In [18]:
df_proportion_percent = spark.sql("SELECT CAST(CEILING(df_percentage*100) AS int) AS int_percentage,* FROM percent")

In [19]:
df_proportion_percent.createOrReplaceTempView("percent")

In [20]:
#spark.sql("SELECT * FROM percent WHERE id == 893353199188").toPandas()

In [21]:
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_april, count, join_key_lookup, row_number, df_percentage]
Index: []

In [22]:
spark.sql("SELECT COUNT(*) FROM percent WHERE CEILING(df_percentage*100) == 60 AND count > 1").limit(10).toPandas()
#6347
#hourly: 4020?

count(1)
0  4020

## add maximum reached percentage to each uuid

In [23]:
# sind auch 1%er dabei
spark.sql("SELECT uuid, MAX(int_percentage) FROM percent WHERE count > 1 GROUP BY uuid ORDER BY 2 DESC").limit(250).toPandas()

uuid  max(int_percentage)
0    f50bc9a02e8346108768fed846dff25c  100                
1    4479cad730344c6a9ecd3e2e7320f58c  100                
2    7524e4a8bc7a4cb4968317b9672c04cd  100                
3    0a6c58220e674e65a01940bd6568b08d  100                
4    892146a606764f7795610045112dcef1  100                
5    99d11310c92648309460da67a2838898  100                
6    88f030b767c645d8b801a4db90ed4b6c  100                
7    46ba6c0b4dbd4f869ae459fad6518fdb  100                
8    2cd3e57424904245919bb340fb13539f  100                
9    a58475f199984aecbf35ad13d91c86f6  100                
10   f1f16112ea9b4ef89573a8c2082b82ac  100                
11   9b39e907a198471a9255ea086be2a6e3  100                
12   c75e54f86dd543e4971b2d9410034401  100                
13   b0c9a6df8c054d06b051360fdee94bfa  100                
14   15f411ada8f6468e96f6a4870fdecff7  100                
15   682e723c7fb448c287a50a33f3690aaa  100                
16   a1a82357933c442786c91e19f5bd0b17  100                
17   340855564dea4a9aa176d9d7eb803234  100                
18   19e7937f830b4771a5b2ee89d7b58017  100                
19   a94463fc688a474db346d2e3ad86512c  100                
20   1f40b518b3134e3ab631309590385048  100                
21   5ddcc0fe8a9341d0be04fa1bac2f97d2  100                
22   5b893c4593344e74b63365364b146df7  100                
23   5134e23f0447407ea9d7b67ef903e6df  100                
24   433324669f1649608b91908a5a0ebe58  100                
25   4fe78ae79a67407f961720282b938dac  100                
26   bea7844684c843aeb139d12d0f6897ac  100                
27   a6f7730184d74997892d5132805790fa  100                
28   07c411a97a0248ca9a82263642fdeed2  100                
29   1cfd9e0002944e938eb6ca38f40134ad  100                
..                                ...  ...                
220  2a579b4df88644c7acb81749a4481dd8  100                
221  b0da200ff1a64c87a825d5f7ecef2f24  100                
222  8961c36dc6ac48878de2cd7c4c52fb07  100                
223  102b8b0a479d4cfdb493bb1ad5f49bf6  100                
224  5f05429e01564f52aba40231cd4f0cd4  100                
225  af070b9f81c14007b853e25bdd5ffa7d  100                
226  00f734a0d6a34db48fc62e7ad5417e80  100                
227  24e406b59cb540e6bac22cded8dbd090  100                
228  307ce10d5ae94711a29548e87bdc3514  100                
229  0e6de01b82594c11b8f99ecedf53faeb  100                
230  524fda5f0c4a450db86ad2c28d4bc4da  100                
231  564ac6350a3646baaf2d659e0c5c791b  100                
232  0802c821cf7548dfbec4d5b73be69563  100                
233  1a944eaab0b94e1fbf4505e392a7829b  100                
234  a6ba741b374f441290f7ba108863cef7  100                
235  4fcf66c5322540f7b260cb3a6a35f8f1  100                
236  78acdc98acd84ca0818a240686082ae8  100                
237  10dd6220fe1a402daa4d339969ab609e  100                
238  638538de8e484c209294a3328e69536a  100                
239  9a4fcc19a8a040809884ad546484fb04  100                
240  141d6452b1754eb1b25dac662ece9d64  100                
241  7dae7221ab6f467e819d7dcfd8da9e52  100                
242  7b6fd5ca24e84075833e8983f4aee661  100                
243  4fb02d58406d414cbd45e3f9dfe27e88  100                
244  2a67146ec2874b76b02ee7b774400295  100                
245  93aeffae62b64307b4ba1c284d2dd7a4  100                
246  7d327144c3cd4c25ac8f186378e8eb6f  100                
247  6663fe30bfff4dc5bda154408e038a6b  100                
248  c8e3998f25ed4765b9148d4f9fd5e25f  100                
249  eef9335af39f42ceb9d9edd52a031196  100                

[250 rows x 2 columns]

In [24]:
# sind auch 1%er dabei
spark.sql("SELECT max, COUNT(*) FROM (SELECT uuid, MAX(int_percentage) AS max FROM percent WHERE count > 1 GROUP BY uuid ORDER BY 2 DESC) GROUP BY max").limit(250).toPandas()

max  count(1)
0   100  3571    
1   99   9       
2   98   10      
3   97   8       
4   96   8       
5   95   11      
6   94   10      
7   93   16      
8   92   5       
9   91   5       
10  90   8       
11  89   11      
12  88   11      
13  87   6       
14  86   6       
15  85   9       
16  84   14      
17  83   3       
18  82   10      
19  81   4       
20  80   16      
21  79   6       
22  78   4       
23  77   6       
24  76   4       
25  75   21      
26  74   8       
27  73   10      
28  72   10      
29  71   5       
..  ..  ..       
70  30   5       
71  29   9       
72  28   9       
73  27   9       
74  26   4       
75  25   47      
76  24   9       
77  23   8       
78  22   9       
79  21   5       
80  20   31      
81  19   7       
82  18   4       
83  17   25      
84  16   12      
85  15   36      
86  14   11      
87  13   20      
88  12   31      
89  11   11      
90  10   46      
91  9    27      
92  8    33      
93  7    29      
94  6    43      
95  5    49      
96  4    74      
97  3    105     
98  2    187     
99  1    827     

[100 rows x 2 columns]

In [25]:
spark.sql("SELECT * FROM percent WHERE uuid == '72c49c0c0e1a459a86dc85e45fe546d7' AND count > 1 AND dataset=='mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00' ORDER BY traceTimeentryUnix ASC").toPandas()

int_percentage  Seq      id   account  accountHash clientState  \
0  50              1    505300  alaperto  1854766604   DONE         
1  100             2    505302  alaperto  1854766604   DONE         

   clientStateHash  \
0 -878527175         
1 -878527175         

                                                                                                                               dataset  \
0  mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   datasetHash datasetScope  datasetScopeHash  \
0  682508441    mc16_13TeV  -1447489496         
1  682508441    mc16_13TeV  -1447489496         

                                            dataset_version  \
0  e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   dataset_versionHash     datatype  datatypeHash eventType  eventTypeHash  \
0 -111191369            NTUP_PILEUP  1967069222    download  59378080        
1 -111191369            NTUP_PILEUP  1967069222    download  59378080        

  eventVersion  eventVersionHash   fileScope  \
0  1.14.11     -2044360925        mc16_13TeV   
1  1.14.11     -2044360925        mc16_13TeV   

                                   filename  filenameHash  filesize_double  \
0  NTUP_PILEUP.12826358._000001.pool.root.1  1272620512    51980.0           
1  NTUP_PILEUP.12826358._000002.pool.root.1 -1764054282    14514.0           

   hits           hostname  hostnameHash           hour    hourHash prod_step  \
0  4     lxplus065.cern.ch  922885429     2018-04-16 12  1356225096  merge      
1  4     lxplus065.cern.ch  922885429     2018-04-16 12  1356225096  merge      

   prod_stepHash     project  projectHash protocol  protocolHash  \
0  874345773      mc16_13TeV -1447489496   gsiftp   2128034789     
1  874345773      mc16_13TeV -1447489496   gsiftp   2128034789     

           remoteSite  remoteSiteHash run_number  run_numberHash   scopeHash  \
0  CERN-PROD_DATADISK -2057286004      363632     336230953      -1447489496   
1  CERN-PROD_DATADISK -2057286004      363632     336230953      -1447489496   

                              stream_name  stream_nameHash  \
0  MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter -740908205         
1  MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter -740908205         

                            traceId  traceIdHash                traceIp  \
0  4d23e3a682d4473fb1e9f7a10c7f66d8  795147204    ::ffff:137.138.52.127   
1  71f0d528720546809c12bd4ce44e76a5 -422568853    ::ffff:137.138.52.127   

   traceIpHash  traceTimeentryUnix  transferStart  \
0  438086758    1.523876e+09        1.523876e+09    
1  438086758    1.523876e+09        1.523876e+09    

                               uuid    uuidHash  \
0  72c49c0c0e1a459a86dc85e45fe546d7 -1055568383   
1  72c49c0c0e1a459a86dc85e45fe546d7 -1055568383   

                                                                                                                                   join_key_april  \
0  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   count  \
0  2       
1  2       

                                                                                                                                  join_key_lookup  \
0  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   row_num

In [26]:
spark.sql("SELECT * FROM percent WHERE uuid == '50288de2738a423f9b650f404e4da890' AND count > 1 ORDER BY int_percentage ASC").toPandas()

int_percentage  Seq      id account  accountHash clientState  \
0    1               1    149241  yliu    1494332205   DONE         
1    1               2    149256  yliu    1494332205   DONE         
2    1               3    149397  yliu    1494332205   DONE         
3    1               1    149929  yliu    1494332205   DONE         
4    1               2    149968  yliu    1494332205   DONE         
5    1               3    149995  yliu    1494332205   DONE         
6    1               1    150106  yliu    1494332205   DONE         
7    1               2    150139  yliu    1494332205   DONE         
8    1               1    150181  yliu    1494332205   DONE         
9    1               1    150779  yliu    1494332205   DONE         
10   1               1    151261  yliu    1494332205   DONE         
11   1               1    152330  yliu    1494332205   DONE         
12   1               2    152604  yliu    1494332205   DONE         
13   1               1    153436  yliu    1494332205   DONE         
14   1               1    161611  yliu    1494332205   DONE         
15   1               1    161655  yliu    1494332205   DONE         
16   1               2    161673  yliu    1494332205   DONE         
17   1               3    161679  yliu    1494332205   DONE         
18   1               1    161687  yliu    1494332205   DONE         
19   1               1    163726  yliu    1494332205   DONE         
20   1               2    164089  yliu    1494332205   DONE         
21   1               1    164533  yliu    1494332205   DONE         
22   1               2    166162  yliu    1494332205   DONE         
23   1               1    166619  yliu    1494332205   DONE         
24   1               2    166969  yliu    1494332205   DONE         
25   1               3    167342  yliu    1494332205   DONE         
26   1               1    170878  yliu    1494332205   DONE         
27   1               1    172547  yliu    1494332205   DONE         
28   1               2    172611  yliu    1494332205   DONE         
29   1               1    173165  yliu    1494332205   DONE         
..  ..              ..       ...   ...           ...    ...         
191  1               1    443086  yliu    1494332205   DONE         
192  1               2    443546  yliu    1494332205   DONE         
193  1               1    444281  yliu    1494332205   DONE         
194  1               1    446689  yliu    1494332205   DONE         
195  1               1    447128  yliu    1494332205   DONE         
196  1               2    447503  yliu    1494332205   DONE         
197  1               1    450704  yliu    1494332205   DONE         
198  1               2    451020  yliu    1494332205   DONE         
199  1               1    451278  yliu    1494332205   DONE         
200  1               2    452155  yliu    1494332205   DONE         
201  1               3    452435  yliu    1494332205   DONE         
202  1               4    452881  yliu    1494332205   DONE         
203  1               1    458858  yliu    1494332205   DONE         
204  1               2    459143  yliu    1494332205   DONE         
205  1               1    461359  yliu    1494332205   DONE         
206  1               1    462580  yliu    1494332205   DONE         
207  1               2    463093  yliu    1494332205   DONE         
208  1               3    463607  yliu    1494332205   DONE         
209  1               1    464450  yliu    1494332205   DONE         
210  1               2    464979  yliu    1494332205   DONE         
211  1               3    465039  yliu    1494332205   DONE         
212  1               1    469208  yliu    1494332205   DONE         
213  1               2    469498  yliu    1494332205   DONE         
214  1               1    470014  yliu    1494332205   DONE         
215  1               2    471294  yliu    1494332205   DONE         
216  1               1    472766  yliu    1494332205   DONE         
217

In [27]:
df_proportion_percent.createOrReplaceTempView("percent")

In [28]:
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, hour, hourHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_april, count, join_key_lookup, row_number, df_percentage]
Index: []

In [29]:
df_proportion_percent_dropped = df_proportion_percent.selectExpr("id as id_proportion", "int_percentage")
df_proportion_percent_dropped.limit(10).toPandas()

id_proportion  int_percentage
0  2              1             
1  4              2             
2  7              1             
3  8              3             
4  9              2             
5  10             2             
6  11             2             
7  14             3             
8  18             3             
9  19             5

In [30]:
#df_final = df_joined_indexed.join(df_proportion_percent_dropped, df_joined_indexed.id == df_proportion_percent_dropped.id_proportion, "left_outer")
df_joined_indexed.createOrReplaceTempView("joined_indexed")
df_proportion_percent_dropped.createOrReplaceTempView("percent_dropped")
df_final = spark.sql("SELECT * FROM joined_indexed LEFT JOIN percent_dropped ON joined_indexed.id == percent_dropped.id_proportion")

In [31]:
df_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE join_key_lookup IS NULL").toPandas()

id   account  accountHash         clientState  clientStateHash  \
0     834     drafanoh -33750993     DONE               -878527175         
1     835     drafanoh -33750993     DONE               -878527175         
2     836     drafanoh -33750993     DONE               -878527175         
3     837     drafanoh -33750993     DONE               -878527175         
4     838     drafanoh -33750993     DONE               -878527175         
5     840     drafanoh -33750993     DONE               -878527175         
6     843     drafanoh -33750993     DONE               -878527175         
7     844     drafanoh -33750993     DONE               -878527175         
8     845     drafanoh -33750993     DONE               -878527175         
9     846     drafanoh -33750993     DONE               -878527175         
10    847     drafanoh -33750993     DONE               -878527175         
11    848     drafanoh -33750993     ServiceUnavailable -1394468120        
12    850     drafanoh -33750993     DONE               -878527175         
13    852     drafanoh -33750993     DONE               -878527175         
14    853     drafanoh -33750993     DONE               -878527175         
15    55952   pgadow    1992700062   DONE               -878527175         
16    59972   drafanoh -33750993     DONE               -878527175         
17    59975   drafanoh -33750993     DONE               -878527175         
18    59977   drafanoh -33750993     DONE               -878527175         
19    154943  jojungge -1078508397   DONE               -878527175         
20    154978  jojungge -1078508397   DONE               -878527175         
21    155362  jojungge -1078508397   DONE               -878527175         
22    155374  jojungge -1078508397   DONE               -878527175         
23    158599  jojungge -1078508397   DONE               -878527175         
24    159553  jojungge -1078508397   DONE               -878527175         
25    159558  jojungge -1078508397   DONE               -878527175         
26    159559  jojungge -1078508397   DONE               -878527175         
27    159596  jojungge -1078508397   DONE               -878527175         
28    159608  jojungge -1078508397   DONE               -878527175         
29    159640  jojungge -1078508397   DONE               -878527175         
...      ...       ...         ...    ...                      ...         
1594  707190  lcorpe   -279559867    DONE               -878527175         
1595  707201  lcorpe   -279559867    DONE               -878527175         
1596  707217  lcorpe   -279559867    DONE               -878527175         
1597  707233  lcorpe   -279559867    DONE               -878527175         
1598  707242  lcorpe   -279559867    DONE               -878527175         
1599  707254  lcorpe   -279559867    DONE               -878527175         
1600  709118  lcorpe   -279559867    DONE               -878527175         
1601  709202  mcfayden -1592538155   DONE               -878527175         
1602  726095  schsu    -927799723    ServiceUnavailable -1394468120        
1603  726120  schsu    -927799723    DONE               -878527175         
1604  735502  goblirsc -305354976    DONE               -878527175         
1605  745375  khamano  -1185418172   DONE               -878527175         
1606  745409  khamano  -1185418172   DONE               -878527175         
1607  745427  khamano  -1185418172   DONE               -878527175         
1608  745436  khamano  -1185418172   DONE               -878527175         
1609  745471  khamano  -1185418172   DONE               -878527175         
1610  745558  khamano  -1185418172   DONE               -878527175         
1611  745567  khamano  -1185418172   DONE               -878527175         
1612  745577  khamano  -1185418172   DONE               -878527175         
1613  745601  khamano  -1185418172   DONE               -878527175         
1614  766780  hegormse -2036406190   DONE               -8

In [32]:
df_final.count()

777286

In [33]:
df_final.sort("traceTimeentryUnix", ascending=True).limit(10).toPandas()

id   account  accountHash         clientState  clientStateHash  \
0  1   yliu      1494332205   ServiceUnavailable -1394468120        
1  2   yliu      1494332205   DONE               -878527175         
2  3   yliu      1494332205   ServiceUnavailable -1394468120        
3  4   changqia  224190018    DONE               -878527175         
4  5   changqia  224190018    ServiceUnavailable -1394468120        
5  6   yliu      1494332205   ServiceUnavailable -1394468120        
6  7   yliu      1494332205   DONE               -878527175         
7  8   heng      99410464     DONE               -878527175         
8  9   yliu      1494332205   DONE               -878527175         
9  10  yliu      1494332205   DONE               -878527175         

                                                                                                                                          dataset  \
0  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
2  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
3  data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
4  data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
5  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
6  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
7  mc16_13TeV.345056.PowhegPythia8EvtGen_NNPDF3_AZNLO_ZH125J_MINLO_vvbb_VpT.deriv.DAOD_HIGG5D1.e5706_e5984_s3126_r9364_r9315_p3260_tid12139571_00   
8  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
9  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           

   datasetHash  datasetScope  datasetScopeHash  \
0 -477538451    data15_13TeV  1217185487         
1 -477538451    data15_13TeV  1217185487         
2 -477538451    data15_13TeV  1217185487         
3 -654722309    data16_13TeV -915656957          
4 -654722309    data16_13TeV -915656957          
5 -477538451    data15_13TeV  1217185487         
6 -477538451    data15_13TeV  1217185487         
7  3665910      mc16_13TeV   -1447489496         
8 -477538451    data15_13TeV  1217185487         
9 -477538451    data15_13TeV  1217185487         

                                      dataset_version  dataset_versionHash  \
0  r9264_p3083_p3213_tid11568050_00                   -762138301             
1  r9264_p3083_p3213_tid11568050_00                   -762138301             
2  r9264_p3083_p3213_tid11568050_00                   -762138301             
3  r9264_p3083_p3372_tid12610361_00                    2032569103            
4  r9264_p3083_p3372_tid12610361_00                    2032569103            
5  r9264_p3083_p3213_tid11568050_00                   -762138301             
6  r9264_p3083_p3213_tid11568050_00                   -762138301             
7  e5706_e5984_s3126_r9364_r9315_p3260_tid12139571_00  747979904             
8  r9264_p3083_p3213_tid11568050_00                   -762138301             
9  r9264_p3083_p3213_tid11568050_00                   -762138301             

       datatype  datatypeHash eventType  eventTypeHash eventVersion  \
0  DAOD_HIGG5D1  1133870593    download  59378080       1.14.11       
1  DAOD_HIGG5D1  1133870593    download  59378080  

In [34]:
df_final = df_final.sort("traceTimeentryUnix", ascending=True)

In [35]:
#df_final.write.format("json").save("/user/sprenner/training_official_april_hourly_proportion.json")

## check if hourly works as expected

In [38]:
df_final.createOrReplaceTempView("final_check")
spark.sql("SELECT uuid,(max-min)/3600 AS diff_hours, from_unixtime(min) AS min, from_unixtime(max) AS max \
           FROM(SELECT uuid, MIN(traceTimeentryUnix) AS min, MAX(traceTimeentryunix) AS max\
           FROM final_check WHERE count > 1 GROUP BY uuid)\
           WHERE min != max AND (max-min)/3600 > 1.0\
           ORDER BY diff_hours DESC").toPandas()

uuid  diff_hours                  min  \
0    606aa9f5a21348d2a0eeb660198d6600  153.882603  2018-04-01 00:38:59   
1    50288de2738a423f9b650f404e4da890  147.628509  2018-04-08 05:06:28   
2    8baa77fc5c2a48e1a555ad11c06d8553  122.022665  2018-04-18 11:54:53   
3    17b1f1dedbac482397aea75d7f7d1c94  120.338976  2018-04-18 13:35:56   
4    bb4fcae1179e4ca89e2796e5a5fc3d65  116.455038  2018-04-23 01:46:26   
5    8b6fef65016d4bf08f15a0e861d4bad3  101.066326  2018-04-19 00:45:15   
6    24390937bd0a4050900c700a727650ea  96.038016   2018-04-12 18:46:43   
7    3f2275aadc1b409abef8a0bb41eaf607  87.982019   2018-04-18 01:55:40   
8    a49c0353bc584e83abbc55ece74b16af  74.986812   2018-04-12 18:12:43   
9    4601b69722da41a189695ad839482940  74.662642   2018-04-27 20:56:46   
10   dda87a65787147a9ba32f15f00ff6082  71.870456   2018-04-11 10:31:26   
11   56075424a6d54596b00bd491cadb0dc3  62.669091   2018-04-21 18:04:04   
12   647da5cfb9ca404b8de47450de7ad6a9  59.970127   2018-04-13 09:07:12   
13   f281c6b71d904302949c11f97d2aee03  58.589340   2018-04-24 08:50:21   
14   b463267f12de4c4f98654c169358cdfd  52.909110   2018-04-08 09:43:23   
15   3e15cf0a38a34d72b185c19756954d31  50.122993   2018-04-06 15:33:54   
16   34868ec92d144675b32f528e6d55afc4  49.353484   2018-04-12 13:30:35   
17   4dbc7ffda6e04ce1a835836b664d159c  48.203906   2018-04-12 16:07:30   
18   68cb93eb562a427e91bef1e617338fb8  46.618259   2018-04-19 05:52:47   
19   43ae524f8eb049e99d48dd6089f59599  44.575892   2018-04-15 21:18:20   
20   7f39ec4282594aff9933ce6b95c8e240  44.442102   2018-04-15 21:25:00   
21   b3250112288b442586b30738ae6b37f2  44.030410   2018-04-12 20:53:46   
22   658f9f7f8bd54a689fb810f337ae3ad8  43.521626   2018-04-19 08:16:33   
23   523e6c129a8245519f3c10e9b7ff35f8  43.053974   2018-04-01 01:48:42   
24   95655dd82e484746bd38c99ae0dc1354  42.160505   2018-04-17 11:09:29   
25   3834b45cac2840b7a4d28c5f559d888e  37.194909   2018-04-10 17:07:02   
26   d654b67d22794069b08baf57f6aea3aa  37.160429   2018-04-10 17:05:04   
27   895eaa69307d444d91d1d38a26629f83  37.064493   2018-04-21 06:23:02   
28   5c8382c8bd894d34a295c07ce886b474  34.663857   2018-04-12 20:55:25   
29   03cbfe4b769a464d97690e2a5f4a4e77  33.928824   2018-04-29 14:04:02   
..                                ...        ...                   ...   
535  5138de77b4de4bf2922f8416040bb045  1.100930    2018-04-19 14:31:36   
536  ee3917ff152048e4a416a4118383d00a  1.093341    2018-04-07 13:22:53   
537  61f786b12629402c92ea27ce7ced88f2  1.091452    2018-04-22 15:59:28   
538  42656f23f6914cb784588f4b58856cdc  1.086197    2018-04-28 23:33:20   
539  e541909f1a794c6fa99350c03ec64932  1.086079    2018-04-11 21:34:45   
540  c17d2f91c0af489fad2cbe1571d2da98  1.085941    2018-04-12 17:27:40   
541  7199922edbec491e8fdfb6f7c9984947  1.085247    2018-04-11 22:39:14   
542  53826e15d7d44d92a5ddd10158448763  1.082190    2018-04-20 18:14:19   
543  aa08c17c2ce04b7ab8ba27d4f1a798fe  1.074163    2018-04-05 22:27:14   
544  8d507ceb54dc4d64b740df076e2ef972  1.073923    2018-04-11 21:52:32   
545  4ac23da1f70344df90820882182b0214  1.072041    2018-04-04 04:07:19   
546  73eb2a9ac54246ffac52a181f27bad29  1.070081    2018-04-11 19:26:11   
547  ba480ce7970e4d7e8c5f9bd03887d3b8  1.065958    2018-04-26 14:33:49   
548  e39383850f0c47cea7708c2dc0381a69  1.061250    2018-04-11 17:05:15   
549  e3428d084ade49978c790f19943caaa4  1.056287    2018-04-11 21:37:14   
550  c8cb1fe0e29f4e7484c4aa4c1cfa6432  1.051696    2018-04-16 00:18:43   
551  ac254208dc53460ea36f52b06f001303  1.048785    2018-04-05 12:31:37   
552  c549db7cb0bf4b5ca5f588a7c2fb2600  1.047007    2018-04-11 20:10:21   
553  3d6bb0451adc4b9eb90cac9616afb36d  1.046991    2018-04-05 20:10:21   
554  01729e73f8044905b321be05d312b180  1.043925    2018-04-07 17:16:39   
555  769a50d7ddd34407a1d8391ead535a80  1.040926    2018-04-11 10:52:02   
556  0c6cbd0eef2f49aaaadcf2225ede634e  1.038461    2018-04-30 17:33:28   
557  39285b4ee6fb403c8ceee9c5e9ea7342

In [39]:
spark.sql("SELECT * FROM final_check").limit(100).toPandas()

id   account  accountHash         clientState  clientStateHash  \
0   1    yliu      1494332205   ServiceUnavailable -1394468120        
1   2    yliu      1494332205   DONE               -878527175         
2   3    yliu      1494332205   ServiceUnavailable -1394468120        
3   4    changqia  224190018    DONE               -878527175         
4   5    changqia  224190018    ServiceUnavailable -1394468120        
5   6    yliu      1494332205   ServiceUnavailable -1394468120        
6   7    yliu      1494332205   DONE               -878527175         
7   8    heng      99410464     DONE               -878527175         
8   9    yliu      1494332205   DONE               -878527175         
9   10   yliu      1494332205   DONE               -878527175         
10  11   yliu      1494332205   DONE               -878527175         
11  12   yliu      1494332205   ServiceUnavailable -1394468120        
12  13   yliu      1494332205   ServiceUnavailable -1394468120        
13  14   yliu      1494332205   DONE               -878527175         
14  15   yliu      1494332205   ServiceUnavailable -1394468120        
15  16   yliu      1494332205   SourceNotFound     -1913538301        
16  17   yliu      1494332205   ServiceUnavailable -1394468120        
17  18   yliu      1494332205   DONE               -878527175         
18  19   heng      99410464     DONE               -878527175         
19  20   yliu      1494332205   ServiceUnavailable -1394468120        
20  21   yliu      1494332205   DONE               -878527175         
21  22   yliu      1494332205   DONE               -878527175         
22  23   yliu      1494332205   DONE               -878527175         
23  24   yliu      1494332205   ServiceUnavailable -1394468120        
24  25   yliu      1494332205   ServiceUnavailable -1394468120        
25  26   yliu      1494332205   ServiceUnavailable -1394468120        
26  27   yliu      1494332205   ServiceUnavailable -1394468120        
27  28   yliu      1494332205   ServiceUnavailable -1394468120        
28  29   changqia  224190018    DONE               -878527175         
29  30   ssolomon  1221542610   ServiceUnavailable -1394468120        
..  ..        ...         ...                  ...         ...        
70  71   changqia  224190018    ServiceUnavailable -1394468120        
71  72   heng      99410464     ServiceUnavailable -1394468120        
72  73   ssolomon  1221542610   ServiceUnavailable -1394468120        
73  74   ssolomon  1221542610   ServiceUnavailable -1394468120        
74  75   ssolomon  1221542610   ServiceUnavailable -1394468120        
75  76   ssolomon  1221542610   DONE               -878527175         
76  77   ssolomon  1221542610   ServiceUnavailable -1394468120        
77  78   ssolomon  1221542610   DONE               -878527175         
78  79   ssolomon  1221542610   ServiceUnavailable -1394468120        
79  80   ssolomon  1221542610   DONE               -878527175         
80  81   ssolomon  1221542610   ServiceUnavailable -1394468120        
81  82   ssolomon  1221542610   DONE               -878527175         
82  83   ssolomon  1221542610   DONE               -878527175         
83  84   ssolomon  1221542610   ServiceUnavailable -1394468120        
84  85   ssolomon  1221542610   ServiceUnavailable -1394468120        
85  86   ssolomon  1221542610   DONE               -878527175         
86  87   ssolomon  1221542610   ServiceUnavailable -1394468120        
87  88   ssolomon  1221542610   DONE               -878527175         
88  89   ssolomon  1221542610   ServiceUnavailable -1394468120        
89  90   ssolomon  1221542610   DONE               -878527175         
90  91   ssolomon  1221542610   ServiceUnavailable -1394468120        
91  92   ssolomon  1221542610   DONE               -878527175         
92  93   ssolomon  1221542610   ServiceUnavailable -1394468120        
93  94   ssolomon  1221542610   DONE               -878527175         
94  95   ssolomon  1221542610

## create table with maximum percent for each hour/uuid/dataset combination

In [42]:
df_total_percent = spark.sql("SELECT hour as hour_total, uuid as uuid_total, join_key_april as join_key_april_total,MAX(int_percentage) as max_percentage FROM final_check GROUP BY hour_total, uuid_total, join_key_april_total")

## join maximum percentages to the corresponding datasets per hour

In [43]:
df_total_percent.createOrReplaceTempView('total')
df_training_prefetching = spark.sql("SELECT * FROM final_check INNER JOIN total ON final_check.hour == total.hour_total \
                                     AND final_check.uuid == total.uuid_total AND final_check.join_key_april == total.join_key_april_total\
                                     SORT BY traceTimeentryUnix ASC")

In [46]:
df_training_prefetching = df_training_prefetching.drop('hour_total','uuid_total','join_key_april_total')

In [53]:
df_training_prefetching.where("uuid == '5138de77b4de4bf2922f8416040bb045' AND join_key_april=='mc16_13TeV:mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00'").toPandas()

id   account  accountHash clientState  clientStateHash  \
0   575641  changqia  224190018    DONE       -878527175         
1   575642  changqia  224190018    DONE       -878527175         
2   575643  changqia  224190018    DONE       -878527175         
3   575644  changqia  224190018    DONE       -878527175         
4   575645  changqia  224190018    DONE       -878527175         
5   575646  changqia  224190018    DONE       -878527175         
6   575647  changqia  224190018    DONE       -878527175         
7   575648  changqia  224190018    DONE       -878527175         
8   575649  changqia  224190018    DONE       -878527175         
9   575650  changqia  224190018    DONE       -878527175         
10  575651  changqia  224190018    DONE       -878527175         
11  575652  changqia  224190018    DONE       -878527175         
12  575653  changqia  224190018    DONE       -878527175         
13  575654  changqia  224190018    DONE       -878527175         
14  575655  changqia  224190018    DONE       -878527175         
15  575656  changqia  224190018    DONE       -878527175         
16  575657  changqia  224190018    DONE       -878527175         
17  575658  changqia  224190018    DONE       -878527175         
18  575659  changqia  224190018    DONE       -878527175         
19  575660  changqia  224190018    DONE       -878527175         
20  575661  changqia  224190018    DONE       -878527175         
21  575662  changqia  224190018    DONE       -878527175         
22  575663  changqia  224190018    DONE       -878527175         
23  575664  changqia  224190018    DONE       -878527175         
24  575665  changqia  224190018    DONE       -878527175         
25  575666  changqia  224190018    DONE       -878527175         
26  575667  changqia  224190018    DONE       -878527175         
27  575668  changqia  224190018    DONE       -878527175         
28  575669  changqia  224190018    DONE       -878527175         
29  575670  changqia  224190018    DONE       -878527175         
30  575671  changqia  224190018    DONE       -878527175         
31  575672  changqia  224190018    DONE       -878527175         
32  575673  changqia  224190018    DONE       -878527175         
33  575674  changqia  224190018    DONE       -878527175         
34  575675  changqia  224190018    DONE       -878527175         
35  575676  changqia  224190018    DONE       -878527175         
36  575677  changqia  224190018    DONE       -878527175         
37  575678  changqia  224190018    DONE       -878527175         
38  575679  changqia  224190018    DONE       -878527175         
39  575680  changqia  224190018    DONE       -878527175         
40  575681  changqia  224190018    DONE       -878527175         
41  575682  changqia  224190018    DONE       -878527175         
42  575685  changqia  224190018    DONE       -878527175         
43  575687  changqia  224190018    DONE       -878527175         

                                                                                                                       dataset  \
0   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00   
1   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00   
2   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00   
3   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00   
4   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00   
5   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00   
6   mc16_13TeV.361600.PowhegPy8EG_CT10nloME_AZNLOCTEQ6L1_WWlvlv.deriv.NTUP_PILEUP.e4616_s3126_r9781_r9778_p3384_tid12739883_00 

In [54]:
df_training_prefetching = df_training_prefetching.sort("traceTimeentryUnix", ascending=True)

In [55]:
df_training_prefetching.write.format("json").save("/user/sprenner/training_official_april_hourly_proportion_prefetching.json")